In [1]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
import matplotlib.pyplot as plt

In [ ]:

# Carregar dados
data = pd.read_csv('precos_acoes.csv')
data['Date'] = pd.to_datetime(data['Date'])

# Vamos utilizar apenas o fechamento das ações para simplificar
close_prices = data['Close'].values.reshape(-1, 1)

# Normalizar dados
scaler = MinMaxScaler()
normalized_data = scaler.fit_transform(close_prices)

# Função para criar sequências de dados
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 10

X, y = create_sequences(normalized_data, SEQ_LENGTH)

# Dividir os dados em treino e teste
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Construir o modelo RNN
model = Sequential()
model.add(SimpleRNN(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Treinar o modelo
model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test))

# Fazer previsões
y_pred = model.predict(X_test)
y_pred_transformed = scaler.inverse_transform(y_pred)
y_test_transformed = scaler.inverse_transform(y_test)

# Avaliar o modelo
mse = mean_squared_error(y_test_transformed, y_pred_transformed)
print(f"Mean Squared Error: {mse}")

# Visualizar previsões vs valores reais
plt.figure(figsize=(10,6))
plt.plot(y_test_transformed, label='True', color='blue')
plt.plot(y_pred_transformed, label='Predicted', color='red')
plt.legend()
plt.show()
